In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os 

from astropy.table import Table

import desispec.io
from desitarget.sv3 import sv3_targetmask 
import astropy

from astropy.cosmology import Planck18 as cosmo

import glob

In [ ]:
specprod = 'fuji'    # Internal name for the EDR
specprod_dir = desispec.io.specprod_root(specprod)
print(specprod_dir)

In [ ]:
bright_zcat_file = os.path.join(specprod_dir, 'zcatalog', 'ztile-sv3-bright-cumulative.fits')

In [ ]:
bright_zcat = Table.read(bright_zcat_file)

In [ ]:
#only bgs bright galaxies
bgs_bright_ii = (bright_zcat['SV3_BGS_TARGET'] & sv3_targetmask.bgs_mask["BGS_BRIGHT"])!=0
print(np.count_nonzero(bgs_bright_ii))
bgs_bright_zcat = bright_zcat[bgs_bright_ii]
print(len(bgs_bright_zcat))

In [ ]:
_ = plt.hist(bgs_bright_zcat['Z'], bins=np.linspace(0.0, 0.8, 20))

In [ ]:
len(set(bgs_bright_zcat['TILEID']))

In [ ]:
# The link between rosette and TILEID is here https://desi.lbl.gov/trac/wiki/SurveyOps/OnePercent#Rosettefootprints

In [ ]:
# here is the file from the LSS catalog
#lss_data = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_BRIGHT_full.dat.fits')
lss_data = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_BRIGHT_N_clustering.dat.fits')

In [ ]:
lss_data

In [ ]:
plt.scatter(lss_data['RA'], lss_data['DEC'])

In [ ]:
list_rosette = np.array(list(set(lss_data['rosette_number'])), dtype='int')

In [ ]:
list_rosette

In [ ]:
# Full description for the rosette footprints.
#https://desi.lbl.gov/trac/wiki/SurveyOps/OnePercent#Rosettefootprints
for rosette_id in list_rosette:
    ii = (lss_data['rosette_number']==rosette_id) & (lss_data['Z']<0.3) & (lss_data['ABSMAG_R']<-20.1826)
    data_rosette = lss_data[ii] 
    print(len(data_rosette), rosette_id)
    plt.figure(figsize=(20,2))
    #plt.subplot(1,2,1)
    #plt.scatter(data_rosette['RA'], data_rosette['DEC'], alpha=0.1)
    #plt.subplot(1,2,2, adjustable='box')

    plt.subplot(1,2,1)
    plt.scatter(data_rosette['Z'], 
                data_rosette['Z']*np.deg2rad(data_rosette['RA']-np.mean(data_rosette['RA'])), s=0.1)
    plt.subplot(1,2,2)
    plt.scatter(data_rosette['RA'], data_rosette['DEC'], s=0.1)
    plt.axis('equal')
    plt.show()

In [ ]:
plt.scatter(lss_data['Z'], lss_data['ABSMAG_R'])

In [ ]:
z_max_list = [0.1, 0.2, 0.25, 0.3, 0.35, 0.4, 0.5]

for z_max in z_max_list:
    ii = lss_data['Z']>z_max
    absmag_r_max = np.max(lss_data['ABSMAG_R'][ii])
    n_in_vol = np.count_nonzero((lss_data['Z']<=z_max) & (lss_data['ABSMAG_R']<absmag_r_max))
    print(z_max, absmag_r_max, n_in_vol)


In [ ]:
# read some random
random  = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_BRIGHT_N_0_clustering.ran.fits')

In [ ]:
random

In [ ]:
list_rosette_random = np.array(list(set(random['rosette_number'])), dtype='int')
print(list_rosette_random)

In [ ]:
# Full description for the rosette footprints.
#https://desi.lbl.gov/trac/wiki/SurveyOps/OnePercent#Rosettefootprints
for rosette_id in list_rosette_random:
    ii = (random['rosette_number']==rosette_id) & (random['Z']<0.3) & (random['ABSMAG_R']<-20.1826)
    data_rosette = random[ii] 
    print(rosette_id, len(data_rosette), len(random))
    plt.figure(figsize=(20,2))
    plt.subplot(1,2,1)
    plt.scatter(data_rosette['Z'], 
                data_rosette['Z']*np.deg2rad(data_rosette['RA']-np.mean(data_rosette['RA'])), s=0.1)
    plt.subplot(1,2,2)
    plt.scatter(data_rosette['RA'], data_rosette['DEC'], s=0.1)
    plt.axis('equal')
    plt.show()

In [ ]:
hemi = 'N'
rand_id = 9
lss_data = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_BRIGHT_{}_clustering.dat.fits'.format(hemi))
random  = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_BRIGHT_{}_{}_clustering.ran.fits'.format(hemi, rand_id))
max_z = 0.3
max_magabs_r = -20.1826
list_rosette = np.array(list(set(lss_data['rosette_number'])), dtype='int')
for rosette_id in list_rosette:
    
    ii = (lss_data['rosette_number']==rosette_id) & (lss_data['Z']<0.3) & (lss_data['ABSMAG_R']<-20.1826)
    data_rosette = lss_data[ii] 
    n_data = len(data_rosette)
    
    ii = (random['rosette_number']==rosette_id) & (random['Z']<0.3) & (random['ABSMAG_R']<-20.1826)
    random_rosette = random[ii]
    n_random = len(random_rosette)
    random_ids = np.arange(n_random)
    select_random_ids = np.random.choice(random_ids, n_data, replace=False)
    random_rosette = random_rosette[select_random_ids]
    
    print(n_data, n_random)

    
    plt.figure(figsize=(20,2))
    plt.subplot(1,2,1)
    plt.scatter(data_rosette['Z'], 
                data_rosette['Z']*np.deg2rad(data_rosette['RA']-np.mean(data_rosette['RA'])), s=0.1)
    plt.scatter(random_rosette['Z'], 
                random_rosette['Z']*np.deg2rad(random_rosette['RA']-np.mean(random_rosette['RA'])), s=0.1, alpha=0.8)    
    plt.subplot(1,2,2)
    plt.scatter(data_rosette['RA'], data_rosette['DEC'], s=0.1)
    plt.scatter(random_rosette['RA'], random_rosette['DEC'], s=0.1, alpha=0.5)
    plt.axis('equal')
    
    file_data = '../data/radec_rosette_{}_data.csv'.format(rosette_id)
    file_random = '../data/radec_rosette_{}_random_{}.csv'.format(rosette_id, rand_id)
    print(file_data, file_random)
    
    astropy.io.ascii.write(random_rosette[['RA', 'DEC', 'Z', 'TARGETID']], file_random, format='csv', overwrite=True)
    astropy.io.ascii.write(data_rosette[['RA', 'DEC', 'Z', 'TARGETID']], file_data, format='csv', overwrite=True)


In [ ]:
def write_data_random(hemi='N', rand_id=0):
    lss_data = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_BRIGHT_{}_clustering.dat.fits'.format(hemi))
    random  = Table.read('/global/cfs/cdirs/desi/survey/catalogs/SV3/LSS/fuji/LSScats/EDAbeta/BGS_BRIGHT_{}_{}_clustering.ran.fits'.format(hemi, rand_id))
    max_z = 0.3
    max_magabs_r = -20.1826
    list_rosette = np.array(list(set(lss_data['rosette_number'])), dtype='int')
    for rosette_id in list_rosette:
    
        ii = (lss_data['rosette_number']==rosette_id) & (lss_data['Z']<0.3) & (lss_data['ABSMAG_R']<-20.1826)
        data_rosette = lss_data[ii] 
        n_data = len(data_rosette)
    
        ii = (random['rosette_number']==rosette_id) & (random['Z']<0.3) & (random['ABSMAG_R']<-20.1826)
        random_rosette = random[ii]
        n_random = len(random_rosette)
        random_ids = np.arange(n_random)
        select_random_ids = np.random.choice(random_ids, n_data, replace=False)
        random_rosette = random_rosette[select_random_ids]
    
        #print(n_data, n_random)

    
        file_data = '../data/radec_rosette_{}_data.csv'.format(rosette_id)
        file_random = '../data/radec_rosette_{}_random_{}.csv'.format(rosette_id, rand_id)
        #print(file_data, file_random)
    
        astropy.io.ascii.write(random_rosette[['RA', 'DEC', 'Z', 'TARGETID']], file_random, format='csv', overwrite=True)
        astropy.io.ascii.write(data_rosette[['RA', 'DEC', 'Z', 'TARGETID']], file_data, format='csv', overwrite=True)


In [ ]:
def convert_radecz_to_xyz(file_in):
    file_out = file_in.replace('radec_', 'xyz_')
    data_in = Table.read(file_in)
    data_in['R'] = np.array(cosmo.comoving_distance(data_in['Z']))
    
    x, y, z = astropy.coordinates.spherical_to_cartesian(data_in['R'], np.array(np.deg2rad(data_in['DEC'])), np.array(np.deg2rad(data_in['RA'])))
    data_in['X'] = x
    data_in['Y'] = y
    data_in['Z'] = z
    #print(file_out)
    #print(data_in)
    #plt.scatter(x,y, s=0.1)
    astropy.io.ascii.write(data_in[['X', 'Y', 'Z', 'TARGETID']], file_out, format='csv', overwrite=True)

In [ ]:
for i in range(15):
    write_data_random(hemi='N', rand_id=i)
    write_data_random(hemi='S', rand_id=i)

In [ ]:
files = glob.glob("../data/radec_*")
for file in files:
    convert_radecz_to_xyz(file)